In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds

from timeit import default_timer as timer

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)

test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size]) 

In [4]:
model_type = "mnist32-cnn_1024_256_64"
model_seed = model_seeds[0]

In [5]:
# Get model
model_instance = model_type + "-" + str(model_seed)
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)

model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [6]:
no_im = 32 # no of images per batch
b_images = test_images[0:no_im]
b_labels = test_labels[0:no_im]
# image = tf.expand_dims(all_images[0],axis=0)

In [7]:
# Load model
model = tf.keras.models.load_model(model_file)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 c0 (Conv2D)                 (None, 29, 29, 32)        544       
                                                                 
 pool2d (MaxPooling2D)       (None, 14, 14, 32)        0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 h0 (Dense)                  (None, 1024)              6423552   
                                                                 
 h1 (Dense)                  (None, 256)               262400    
                                                                 
 h2 (Dense)                  (None, 64)                16448     
                                                                 
 op (Dense)                  (None, 10)                6

In [8]:
# testing batch_mnist32_cnn_ERR
from libs.fitnessfns import batch_mnist32_cnn_ERR
error_profile_c0 = None
error_profile_h0 = None
error_profile_h1 = None
error_profile_h2 = None
error_profile_op = None
ERR_PARAM_TF = None
shuffle_order_c0 = None
shuffle_order_h0 = None
shuffle_order_h1 = None
shuffle_order_h2 = None
shuffle_order_op = None
                                        
manual_output = batch_mnist32_cnn_ERR(  b_images,
                                        b_labels,
                                        model,
                                        error_profile_c0,
                                        error_profile_h0,
                                        error_profile_h1,
                                        error_profile_h2,
                                        error_profile_op,
                                        ERR_PARAM_TF,
                                        shuffle_order_c0,
                                        shuffle_order_h0,
                                        shuffle_order_h1,
                                        shuffle_order_h2,
                                        shuffle_order_op)

print(manual_output)

tf.Tensor(0, shape=(), dtype=int64)


In [9]:
# testing ff_mnist32_cnn_ERR
from libs.fitnessfns import ff_mnist32_cnn_ERR
error_profile_c0 = None
error_profile_h0 = None
error_profile_h1 = None
error_profile_h2 = None
error_profile_op = None
ERR_PARAM = None
shuffle_order_c0 = None
shuffle_order_h0 = None
shuffle_order_h1 = None
shuffle_order_h2 = None
shuffle_order_op = None
test_set = (test_images, test_labels)
batchsize = 128
                                        
acc = ff_mnist32_cnn_ERR(model,
                        error_profile_c0,
                        error_profile_h0,
                        error_profile_h1,
                        error_profile_h2,
                        error_profile_op,
                        ERR_PARAM,
                        shuffle_order_c0,
                        shuffle_order_h0,
                        shuffle_order_h1,
                        shuffle_order_h2,
                        shuffle_order_op,
                        test_set,
                        batchsize)

print(acc)

tf.Tensor(0.9859775641025641, shape=(), dtype=float64)


In [10]:
# testing eval_mnist32_cnn_ERR
from libs.fitnessfns import eval_mnist32_cnn_ERR

error_profile_c0 = None
error_profile_h0 = None
error_profile_h1 = None
error_profile_h2 = None
error_profile_op = None
ERR_PARAM = None
shuffle_order_c0 = None
shuffle_order_h0 = None
shuffle_order_h1 = None
shuffle_order_h2 = None
shuffle_order_op = None
test_set = (test_images, test_labels)

acc_mean, acc_std = eval_mnist32_cnn_ERR(model,
                        error_profile_c0,
                        error_profile_h0,
                        error_profile_h1,
                        error_profile_h2,
                        error_profile_op,
                        ERR_PARAM,
                        shuffle_order_c0,
                        shuffle_order_h0,
                        shuffle_order_h1,
                        shuffle_order_h2,
                        shuffle_order_op,
                        test_set)

print(acc_mean, acc_std)

0.9859775641025642 1.1102230246251565e-16


In [11]:
# testing eval_mnist32_cnn_ERR with dummy error profile and shuffle order
dummy_error_profile = np.zeros((20_000,32), dtype="float32")
shuffle_order_c0 = np.arange(32)
shuffle_order_h0 = np.arange(1024)
shuffle_order_h1 = np.arange(256)
shuffle_order_h2 = np.arange(64)
shuffle_order_op = np.arange(10)



error_profile_c0 = dummy_error_profile
error_profile_h0 = dummy_error_profile
error_profile_h1 = dummy_error_profile
error_profile_h2 = dummy_error_profile
error_profile_op = dummy_error_profile
ERR_PARAM = -1
np.random.shuffle(shuffle_order_c0)
np.random.shuffle(shuffle_order_h0)
np.random.shuffle(shuffle_order_h1)
np.random.shuffle(shuffle_order_h2)
np.random.shuffle(shuffle_order_op)
test_set = (test_images, test_labels)

acc_mean, acc_std = eval_mnist32_cnn_ERR(model,
                        error_profile_c0,
                        error_profile_h0,
                        error_profile_h1,
                        error_profile_h2,
                        error_profile_op,
                        ERR_PARAM,
                        shuffle_order_c0,
                        shuffle_order_h0,
                        shuffle_order_h1,
                        shuffle_order_h2,
                        shuffle_order_op,
                        test_set)

print(acc_mean, acc_std)

0.9860777243589743 0.0
